In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pipulate import pip
import secretsauce
import nest_asyncio
import keys
nest_asyncio.apply()

job = "gapalyzer-01" # Give your session a unique name

# Here are the Keys

In [ ]:

pip.api_key(job, key=keys.google)
botify_token = keys.botify


## Here are your Foes

### Save all of These

In [ ]:
import nbformat
from pathlib import Path

def get_competitors_from_notebook(notebook_filename="GAPalyzer.ipynb"):
    """Parses this notebook to get the domain list from the 'url-list-input' cell."""
    try:
        notebook_path = Path(notebook_filename)
        with open(notebook_path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
        
        for cell in nb.cells:
            if "url-list-input" in cell.metadata.get("tags", []):
                domains_raw = cell.source
                domains = [
                    line.split('#')[0].strip() 
                    for line in domains_raw.splitlines() 
                    if line.strip() and not line.strip().startswith('#')
                ]
                return domains
        print("⚠️ Warning: Could not find a cell tagged with 'url-list-input'.")
        return []
    except Exception as e:
        print(f"❌ Error reading domains from notebook: {e}")
        return []

# --- Main Logic ---
print("🚀 Generating SEMrush URLs for GAP analysis...")

domains = get_competitors_from_notebook()
url_template = "https://www.semrush.com/analytics/organic/positions/?db=us&q={domain}&searchType=domain"

if not domains:
    print("🛑 No domains found. Please add competitor domains to the 'url-list-input' cell and re-run.")
else:
    print(f"✅ Found {len(domains)} competitor domains. Click the links below to open each report:")
    print("-" * 30)
    for i, domain in enumerate(domains):
        full_url = url_template.format(domain=domain)
        print(f"{i+1}. {domain}:\n   {full_url}\n")

### Stack 'Em, Join 'Em & Tag 'Em! 🥞🧠📊
#### 1. Stack 'Em (turns all downloads into 1)

In [ ]:
import pandas as pd
from pathlib import Path
import os
import nbformat # Make sure this import is here

def get_all_domains_from_notebook(notebook_filename="GAPalyzer.ipynb"):
    """Parses this notebook to get the full domain list from the 'url-list-input' cell."""
    try:
        notebook_path = Path(notebook_filename)
        with open(notebook_path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
        
        for cell in nb.cells:
            if "url-list-input" in cell.metadata.get("tags", []):
                domains_raw = cell.source
                domains = [
                    line.split('#')[0].strip() 
                    for line in domains_raw.splitlines() 
                    if line.strip() and not line.strip().startswith('#')
                ]
                return domains
        print("⚠️ Warning: Could not find a cell tagged with 'url-list-input'.")
        return []
    except Exception as e:
        print(f"❌ Error reading domains from notebook: {e}")
        return []

# --- Main Logic ---
print("🥞 Stacking 'Em: Consolidating all SEMrush Excel files...")

# 1. Get all domains from the notebook cell
all_domains = get_all_domains_from_notebook()

if not all_domains:
    print("🛑 No domains found. Please add your client and competitor domains to the 'url-list-input' cell.")
else:
    # 2. THE NEW CONVENTION: The first domain is always the client's domain.
    your_domain = all_domains[0]
    print(f"✅ Client domain identified as: {your_domain}")
    
    downloads_dir = Path.home() / "Downloads"
    all_excel_files = list(downloads_dir.glob("*.xlsx"))
    
    list_of_dataframes = []
    for file_path in all_excel_files:
        try:
            domain_name = file_path.name.split('-organic.Positions')[0]
            # Only process files that are in our domain list
            if domain_name in all_domains:
                df = pd.read_excel(file_path)
                df['domain'] = domain_name
                list_of_dataframes.append(df)
                print(f"  -> ✅ Loaded and tagged {file_path.name}")
        except Exception as e:
            print(f"  -> ⚠️ Could not process file {file_path.name}: {e}")

    if list_of_dataframes:
        combined_df = pd.concat(list_of_dataframes, ignore_index=True)
        print("\n🎉 Success! All relevant Excel files have been consolidated.")
        display(combined_df.head())
    else:
        print("🤷 No matching Excel files were found in the Downloads folder.")

### 2. Join 'em

In [ ]:
import pandas as pd

print("🧩 Joining 'Em: Pivoting data to map keywords against competitors...")

if 'combined_df' not in locals() or 'your_domain' not in locals():
    print("❌ 'combined_df' or 'your_domain' not found. Please run the 'Stack \'Em' cell first.")
else:
    try:
        pivot_data = combined_df[['Keyword', 'domain', 'Search Volume']]
        pivoted_df = pivot_data.pivot_table(
            index='Keyword', 
            columns='domain', 
            values='Search Volume',
            aggfunc='first'
        ).fillna(0)

        # Reorder columns to put the client's domain first
        if your_domain in pivoted_df.columns:
            cols = [your_domain] + [col for col in pivoted_df if col != your_domain]
            pivoted_df = pivoted_df[cols]
        else:
            print(f"⚠️ Warning: Your specified domain '{your_domain}' was not found in the data.")

        print("\n🎉 Success! Data has been pivoted into a competitive matrix.")
        display(pivoted_df.head())

    except Exception as e:
        print(f"\n❌ An error occurred during the pivot process: {e}")

In [ ]:
import pandas as pd

print("🏷️ Tagging 'Em: Identifying and scoring content gaps...")

if 'pivoted_df' not in locals() or 'combined_df' not in locals() or 'your_domain' not in locals():
    print("❌ Required DataFrames or 'your_domain' variable not found. Please run the previous cells first.")
else:
    try:
        # Merge Search Volume back into the pivoted table
        search_volumes = combined_df[['Keyword', 'Search Volume']].drop_duplicates().set_index('Keyword')
        analysis_df = pivoted_df.merge(search_volumes, left_index=True, right_index=True)
        
        # Identify competitor columns
        competitor_cols = [col for col in pivoted_df.columns if col != your_domain]
        
        # Count competitor rankings
        analysis_df['competitor_count'] = (analysis_df[competitor_cols] > 0).sum(axis=1)
        
        # Find the gaps where your domain ranks 0
        gap_df = analysis_df[analysis_df[your_domain] == 0].copy()
        
        # Score the opportunity
        gap_df['opportunity_score'] = gap_df['Search Volume'] * gap_df['competitor_count']
        
        # Sort by the highest opportunity score
        gap_df_sorted = gap_df.sort_values(by='opportunity_score', ascending=False)
        
        print("\n🎉 Success! Content gaps have been identified and scored.")
        print("-" * 30)
        
        print("Top 20 Content Opportunities:")
        display(gap_df_sorted.head(20))

    except Exception as e:
        print(f"\n❌ An error occurred during the 'Tag \'Em' process: {e}")

### Step 4: Export to a Beautifully Formatted Excel File 🎨

In [ ]:
import pandas as pd
from pathlib import Path
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment
from openpyxl.worksheet.table import Table, TableStyleInfo
import ipywidgets as widgets
from IPython.display import display
import platform
import subprocess
import os

def export_gap_analysis_to_excel(job: str, df: pd.DataFrame):
    """
    Exports the GAPalyzer DataFrame to a professionally formatted Excel file
    inside the 'output' folder.
    """
    if df.empty:
        print("⚠️ DataFrame is empty, skipping file export.")
        return

    output_dir = Path("output")
    output_dir.mkdir(parents=True, exist_ok=True)

    excel_filename = output_dir / f"{job}_gap_analysis.xlsx"
    
    print(f"🎨 Formatting and exporting data to Excel: {excel_filename}")
    
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='GAP_Analysis')
        
        worksheet = writer.sheets['GAP_Analysis']

        # 1. Create an Excel Table for banded rows and filtering
        table_range = f"A1:{get_column_letter(worksheet.max_column)}{worksheet.max_row}"
        table = Table(displayName="GapAnalysisTable", ref=table_range)
        style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                               showLastColumn=False, showRowStripes=True, showColumnStripes=False)
        table.tableStyleInfo = style
        worksheet.add_table(table)

        # 2. Define consistent column widths
        # We'll make educated guesses for the columns from your pivot
        width_map = {
            "Keyword": 50,
            "Search Volume": 18,
            "competitor_count": 18,
            "opportunity_score": 18,
        }
        default_width = 15 # For competitor domains

        # 3. Apply formatting
        for col_idx, column_cell in enumerate(worksheet[1], 1):
            column_letter = get_column_letter(col_idx)
            header_text = str(column_cell.value)
            
            # A. Format header cell
            column_cell.font = Font(bold=True)
            column_cell.alignment = Alignment(horizontal='center', vertical='center')

            # B. Set column width
            width = width_map.get(header_text, default_width)
            worksheet.column_dimensions[column_letter].width = width

        # 4. Apply text wrapping and vertical alignment to all data cells
        for row in worksheet.iter_rows(min_row=2):
            for cell in row:
                cell.alignment = Alignment(wrap_text=True, vertical='top')

    print(f"✅ Success! Files saved in the '{output_dir}' folder.")
    
    # --- Helper to open the output folder ---
    def _open_folder(path_str: str = "."):
        folder_path = Path(path_str).resolve()
        print(f"Attempting to open folder: {folder_path}")
        if not folder_path.exists() or not folder_path.is_dir():
            print(f"❌ Error: Path is not a valid directory: {folder_path}")
            return
        system = platform.system()
        try:
            if system == "Windows": os.startfile(folder_path)
            elif system == "Darwin": subprocess.run(["open", folder_path])
            else: subprocess.run(["xdg-open", folder_path])
        except Exception as e:
            print(f"❌ Failed to open folder. Please navigate to it manually. Error: {e}")

    # --- Display the "Open Folder" button ---
    button = widgets.Button(
        description="📂 Open Output Folder",
        tooltip=f"Open {output_dir.resolve()}",
        button_style='success'
    )
    button.on_click(lambda b: _open_folder("output"))
    display(button)

# --- Main Logic ---
if 'gap_df_sorted' not in locals():
    print("❌ 'gap_df_sorted' not found. Please run the 'Tag \'Em' cell first.")
else:
    # Call the export function
    export_gap_analysis_to_excel(job, gap_df_sorted)

# Todo
- Make it work with Excel or CSV files
- Bring your own CSV (Botify, GSC, etc) for joined rows & PageRank and other signals
- A way to take ad hoc metrics into account with opportunity scores
- Botify API integration in place of bring your own CSV